In [2]:
# !~/tf-nvidia/bin/pip3 install git+https://github.com/huseinzol05/tensor2tensor.git --no-deps

In [3]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [4]:
from tensor2tensor import models
from tensor2tensor import problems
from tensor2tensor.layers import common_layers
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import t2t_model
from tensor2tensor.utils import registry
from tensor2tensor.utils import metrics
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.data_generators import translate
from tensor2tensor.utils import registry

2022-07-05 21:41:15.372280: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0



/home/ubuntu/tf-nvidia/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from malaya.text.t2t import text_encoder
import malaya

In [6]:
encoder = text_encoder.SubwordTextEncoder('ms-en.subwords')

In [7]:
class Encoder:
    def __init__(self, encoder):
        self.encoder = encoder
        self.vocab_size = encoder.vocab_size

    def encode(self, s):
        s = [self.encoder.encode(s_) for s_ in s]
        s = [i + [1] for i in s]
        return s

    def decode(self, ids, strip_extraneous = False):
        return self.encoder.decode(ids)

In [8]:
s_encoder = Encoder(encoder)

In [9]:
@registry.register_problem
class Translation(text_problems.Text2TextProblem):
    @property
    def approx_vocab_size(self):
        return encoder.vocab_size

    @property
    def is_generate_per_split(self):
        return False

    @property
    def dataset_splits(self):
        return [
            {'split': problem.DatasetSplit.TRAIN, 'shards': 100},
            {'split': problem.DatasetSplit.EVAL, 'shards': 1},
        ]

    def feature_encoders(self, data_dir):
        s_encoder = Encoder(encoder)
        return {'inputs': s_encoder, 'targets': s_encoder}

In [10]:
PROBLEM = 'translation'
problem = problems.problem(PROBLEM)

In [11]:
class Model:
    def __init__(self, HPARAMS = "transformer_base", DATA_DIR = 't2t/data2'):
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])
        self.beam_size = tf.placeholder(tf.int32, shape=(), name = 'beam_size')
        self.temperature = tf.placeholder(tf.float32, shape=(), name = 'temperature')
        
        self.X_seq_len = tf.count_nonzero(self.X, 1, dtype=tf.int32)
        maxlen_decode = 50 + tf.reduce_max(self.X_seq_len)
        
        x = tf.expand_dims(tf.expand_dims(self.X, -1), -1)
        y = tf.expand_dims(tf.expand_dims(self.Y, -1), -1)
        
        features = {
            "inputs": x,
            "targets": y,
            "target_space_id": tf.constant(1, dtype=tf.int32),
        }
        print(features)
        
        Modes = tf.estimator.ModeKeys
        hparams = trainer_lib.create_hparams(HPARAMS, data_dir=DATA_DIR, problem_name=PROBLEM)
        translate_model = registry.model('transformer')(hparams, Modes.PREDICT)
        logits, _ = translate_model(features)
        
        with tf.variable_scope(tf.get_variable_scope(), reuse=True):
            self.fast_result = translate_model._greedy_infer(features, maxlen_decode)["outputs"]
            self.beam_result = translate_model._beam_decode_slow(
                features, maxlen_decode, beam_size=self.beam_size, 
                top_beams=1, alpha=self.temperature)["outputs"]
        
        self.fast_result = tf.identity(self.fast_result, name = 'greedy')
        self.beam_result = tf.identity(self.beam_result, name = 'beam')

In [12]:
import tensorflow as tf

In [13]:
ckpt_path = tf.train.latest_checkpoint('t2t-noisy-ms-en/base')
ckpt_path

't2t-noisy-ms-en/base/model.ckpt-675000'

In [14]:
!ls t2t-noisy-ms-en/base/

checkpoint
eval
events.out.tfevents.1656740328.huseincomel-desktop
events.out.tfevents.1656841158.huseincomel-desktop
events.out.tfevents.1656864889.huseincomel-desktop
events.out.tfevents.1656946953.huseincomel-desktop
graph.pbtxt
model.ckpt-200000.data-00000-of-00002
model.ckpt-200000.data-00001-of-00002
model.ckpt-200000.index
model.ckpt-200000.meta
model.ckpt-225000.data-00000-of-00002
model.ckpt-225000.data-00001-of-00002
model.ckpt-225000.index
model.ckpt-225000.meta
model.ckpt-250000.data-00000-of-00002
model.ckpt-250000.data-00001-of-00002
model.ckpt-250000.index
model.ckpt-250000.meta
model.ckpt-275000.data-00000-of-00002
model.ckpt-275000.data-00001-of-00002
model.ckpt-275000.index
model.ckpt-275000.meta
model.ckpt-300000.data-00000-of-00002
model.ckpt-300000.data-00001-of-00002
model.ckpt-300000.index
model.ckpt-300000.meta
model.ckpt-325000.data-00000-of-00002
model.ckpt-325000.data-00001-of-00002
model.ckpt-325000.index
model.ckpt-325000.meta
model.ckpt-350000.data-00000-o

In [15]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model()

2022-07-05 21:41:39.468001: I tensorflow/core/platform/profile_utils/cpu_utils.cc:109] CPU Frequency: 2496000000 Hz
2022-07-05 21:41:39.468391: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x8a94420 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-07-05 21:41:39.468413: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-07-05 21:41:39.469480: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-07-05 21:41:39.472550: E tensorflow/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-07-05 21:41:39.472565: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: huseincomel-desktop
2022-07-05 21:41:39.472578: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: huseincomel

{'inputs': <tf.Tensor 'ExpandDims_1:0' shape=(?, ?, 1, 1) dtype=int32>, 'targets': <tf.Tensor 'ExpandDims_3:0' shape=(?, ?, 1, 1) dtype=int32>, 'target_space_id': <tf.Tensor 'Const_1:0' shape=() dtype=int32>}
INFO:tensorflow:Setting T2TModel mode to 'infer'


INFO:tensorflow:Setting T2TModel mode to 'infer'


INFO:tensorflow:Setting hparams.dropout to 0.0


INFO:tensorflow:Setting hparams.dropout to 0.0


...

INFO:tensorflow:Transforming feature 'targets' with symbol_modality_25880_512.targets_bottom


INFO:tensorflow:Building model body


INFO:tensorflow:Building model body


INFO:tensorflow:Transforming body output with symbol_modality_25880_512.top


INFO:tensorflow:Transforming body output with symbol_modality_25880_512.top


In [21]:
var_lists = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
saver = tf.train.Saver(var_list = var_lists)
saver.restore(sess, 't2t-noisy-ms-en/base/model.ckpt-625000')

INFO:tensorflow:Restoring parameters from t2t-noisy-ms-en/base/model.ckpt-625000


INFO:tensorflow:Restoring parameters from t2t-noisy-ms-en/base/model.ckpt-625000


In [22]:
string = 'sedangkan,Xpernah pon kita kacau solat n penampilan orang lain.'
encoded = encoder.encode(string) + [1]
f, b = sess.run([model.fast_result, model.beam_result], feed_dict = {model.X: [encoded], model.temperature: 0.5,
                                                                    model.beam_size: 2})
encoder.decode(f[0]), encoder.decode(b[0])

('whereas, we do not even disturb the prayers and appearance of others.<EOS>',
 'whereas, we do not even disturb the prayers and appearance of others.<EOS>')

In [23]:
string = 'Agama pada lazimnya bermakna kepercayaan kepada Tuhan, atau sesuatu kuasa yang ghaib dan sakti seperti Dewa, dan juga amalan dan institusi yang berkait dengan kepercayaan tersebut. Agama dan kepercayaan merupakan dua pekara yang sangat berkaitan. Tetapi Agama mempunyai makna yang lebih luas, yakni merujuk kepada satu sistem kepercayaan yang kohensif, dan kepercayaan ini adalah mengenai aspek ketuhanan.'
encoded = encoder.encode(string) + [1]
f, b = sess.run([model.fast_result, model.beam_result], feed_dict = {model.X: [encoded],model.temperature: 0.5,
                                                                    model.beam_size: 2})
encoder.decode(f[0]), encoder.decode(b[0])

('Religion usually means belief in God, or a supernatural and supernatural power such as God, as well as the practices and institutions associated with that belief. Religion and belief are two very related things. But Religion has a broader meaning, referring to a cohensive belief system, and this belief is about the divine aspect.<EOS>',
 'Religion usually means belief in God, or a supernatural and supernatural power such as God, as well as the practices and institutions associated with that belief. Religion and belief are two very related things. But Religion has a broader meaning, referring to a cohensive belief system, and this belief is about the divine aspect.<EOS>')

In [24]:
string = 'ak tak paham la'
encoded = encoder.encode(string) + [1]
f, b = sess.run([model.fast_result, model.beam_result], feed_dict = {model.X: [encoded],model.temperature: 0.5,
                                                                    model.beam_size: 2})
encoder.decode(f[0]), encoder.decode(b[0])

("I don't understand.<EOS>", "I don't understand.<EOS>")

In [25]:
string = 'jam 8 di pasar KK memang org ramai 😂 pandai dia pilih tmpt.'
encoded = encoder.encode(string) + [1]
f, b = sess.run([model.fast_result, model.beam_result], feed_dict = {model.X: [encoded],model.temperature: 0.5,
                                                                    model.beam_size: 2})
encoder.decode(f[0]), encoder.decode(b[0])

("At 8 in the KK market there are people 😂 he's good at choosing a place.<EOS>",
 "At 8 in the KK market there are people 😂 he's good at choosing a place.<EOS>")

In [26]:
string = "Teoh is actually a more accurate representation of Najib's base support than his Bossku fans. Members of an elite ecosystem of cronies and conmen who could only ever get by through the networks a white-collar criminal like him can provide."
encoded = encoder.encode(string) + [1]
f, b = sess.run([model.fast_result, model.beam_result], feed_dict = {model.X: [encoded],model.temperature: 0.5,
                                                                    model.beam_size: 2})
encoder.decode(f[0]), encoder.decode(b[0])

("Teoh is actually a more accurate representation of Najib's base support than his Bossku fans. Members of an elite ecosystem of cronies and conmen who could only ever get by through the networks a white-collar criminal like him can provide.<EOS>",
 "Teoh is actually a more accurate representation of Najib's base support than his Bossku fans. Members of an elite ecosystem of cronies and conmen who could only ever get by through the networks a white-collar criminal like him can provide.<EOS>")

In [27]:
string = 'Jadi haram jadah😀😃🤭'
encoded = encoder.encode(string) + [1]
f, b = sess.run([model.fast_result, model.beam_result], feed_dict = {model.X: [encoded],model.temperature: 0.5,
                                                                    model.beam_size: 2})
encoder.decode(f[0]), encoder.decode(b[0])

('So illegal make😀😃🤭<EOS>', 'So illegal make😀😃🤭<EOS><pad><pad><pad><pad>')

In [29]:
string = 'nak gi mana tuu'
encoded = encoder.encode(string) + [1]
f, b = sess.run([model.fast_result, model.beam_result], feed_dict = {model.X: [encoded],model.temperature: 0.5,
                                                                    model.beam_size: 2})
encoder.decode(f[0]), encoder.decode(b[0])

('Where is that?<EOS>', 'Where is that?<EOS>')

In [30]:
strings = ['Since aku nak expand db beyond rds',
           'Ada macam blueprint tak untuk data team ni',
           'Macam nak ambil half day',
           "Bayangkan PH dan menang pru-14. Pastu macam-macam pintu belakang ada. Last-last Ismail Sabri naik. That's why I don't give a fk about politics anymore. Sumpah dah fk up dah.",
           'aku kena tidoq, 4 pagi ada migration',
           'Cein. Kalau aku nak tubuhkan data team kat company sekarang ni, apa job scope dia and apa position baru yang akan wujud']

In [31]:
for string in strings:
    encoded = encoder.encode(string) + [1]
    f, b = sess.run([model.fast_result, model.beam_result], feed_dict = {model.X: [encoded],model.temperature: 0.5,
                                                                        model.beam_size: 2})
    print(encoder.decode(f[0]), encoder.decode(b[0]))

Since I want to expand db beyond independence.<EOS> Since I want to expand db beyond independence.<EOS><pad>
There's a kind of blueprint not for this team data.<EOS> There's kind of blueprint not for this team data.<EOS><pad>
It's like taking half a day.<EOS> It's like taking half a day.<EOS>
Imagine PH and won 14. Then there are all kinds of back doors. Ismail Sabri's last time went up. That's why I don't give a fk about politics anymore. The oath has been poured up.<EOS> Imagine PH and won 14. Then there are all kinds of back doors available. Ismail Sabri's late-finals are up. That's why I don't give a fk about politics anymore. The oath has already gone up.<EOS><pad><pad><pad>
I got tidoq, 4 in the morning there was a migration.<EOS> I got tidoq, 4 in the morning there was a migration.<EOS>
I want to set up team data with the company right now, what is the job scope and what a new position it will be about?<EOS> I want to set up team data with the company right now, what is the job 

In [55]:
batch_size = 16

path = 'test'

with open(os.path.join(path, 'left.txt')) as fopen:
    left = fopen.read().split('\n')
    
with open(os.path.join(path, 'right.txt')) as fopen:
    right = fopen.read().split('\n')
    
len(left), len(right)

(100000, 100000)

In [56]:
p = sess.run(model.fast_result, feed_dict = {model.X: [encoder.encode(left[0]) + [1]]}).tolist()
results = []
for row in p:
    results.append([i for i in row if i not in [0, 1]])
results

[[6174,
  13394,
  21952,
  156,
  10,
  442,
  10,
  2622,
  7335,
  13,
  6174,
  13394,
  21952,
  156,
  10,
  442,
  10,
  2622,
  7335,
  13,
  14,
  11,
  389,
  9,
  4,
  10178,
  514,
  5,
  4,
  12805,
  10,
  2955,
  538,
  5,
  4167,
  407,
  3]]

In [57]:
from tensor2tensor.utils import bleu_hook

In [58]:
bleu_hook.compute_bleu(reference_corpus = [encoder.encode(right[0])], 
                       translation_corpus = results)

1.0

In [59]:
pad_sequences = tf.keras.preprocessing.sequence.pad_sequences

In [ ]:
from tqdm import tqdm

results = []
for i in tqdm(range(0, len(left), batch_size)):
    index = min(i + batch_size, len(left))
    x = left[i: index]
    encoded = [encoder.encode(l) + [1] for l in x]
    batch_x = pad_sequences(encoded, padding='post')
    
    p = sess.run(model.fast_result, feed_dict = {model.X: batch_x}).tolist()
    result = []
    for row in p:
        result.append([i for i in row if i not in [0, 1]])
    results.extend(result)

In [32]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'transformer-base/model.ckpt')

'transformer-base/model.ckpt'

In [33]:
strings = ','.join(
    [
        n.name
        for n in tf.get_default_graph().as_graph_def().node
        if ('Variable' in n.op
        or 'Placeholder' in n.name
        or 'greedy' in n.name
        or 'beam' in n.name
        or 'alphas' in n.name
        or 'temperature' in n.name
        or 'beam_size' in n.name
        or 'self/Softmax' in n.name)
        and 'adam' not in n.name
        and 'beta' not in n.name
        and 'global_step' not in n.name
        and 'modality' not in n.name
        and 'Assign' not in n.name
    ]
)
strings.split(',')

['Placeholder',
 'Placeholder_1',
 'beam_size',
 'temperature',
 'transformer/body/target_space_embedding/kernel/Read/ReadVariableOp',
 'transformer/parallel_0_4/transformer/transformer/body/target_space_embedding/ReadVariableOp',
 'transformer/body/encoder/layer_0/self_attention/layer_prepostprocess/layer_norm/layer_norm_scale/Read/ReadVariableOp',
 'transformer/body/encoder/layer_0/self_attention/layer_prepostprocess/layer_norm/layer_norm_bias/Read/ReadVariableOp',
 'transformer/parallel_0_4/transformer/transformer/body/encoder/layer_0/self_attention/layer_prepostprocess/layer_norm/ReadVariableOp',
 'transformer/parallel_0_4/transformer/transformer/body/encoder/layer_0/self_attention/layer_prepostprocess/layer_norm/ReadVariableOp_1',
 'transformer/body/encoder/layer_0/self_attention/multihead_attention/q/kernel/Read/ReadVariableOp',
 'transformer/parallel_0_4/transformer/transformer/body/encoder/layer_0/self_attention/multihead_attention/q/Tensordot/ReadVariableOp',
 'transformer/bod

In [34]:
def freeze_graph(model_dir, output_node_names):

    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            'directory: %s' % model_dir
        )

    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path

    absolute_model_dir = '/'.join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + '/frozen_model.pb'
    clear_devices = True
    with tf.Session(graph = tf.Graph()) as sess:
        saver = tf.train.import_meta_graph(
            input_checkpoint + '.meta', clear_devices = clear_devices
        )
        saver.restore(sess, input_checkpoint)
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            tf.get_default_graph().as_graph_def(),
            output_node_names.split(','),
        )
        with tf.gfile.GFile(output_graph, 'wb') as f:
            f.write(output_graph_def.SerializeToString())
        print('%d ops in the final graph.' % len(output_graph_def.node))

In [35]:
freeze_graph('transformer-base', strings)

INFO:tensorflow:Restoring parameters from transformer-base/model.ckpt


INFO:tensorflow:Restoring parameters from transformer-base/model.ckpt


INFO:tensorflow:Froze 201 variables.


INFO:tensorflow:Froze 201 variables.


INFO:tensorflow:Converted 201 variables to const ops.


INFO:tensorflow:Converted 201 variables to const ops.


11017 ops in the final graph.


In [36]:
def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph

In [37]:
g = load_graph('transformer-base/frozen_model.pb')
x = g.get_tensor_by_name('import/Placeholder:0')
greedy = g.get_tensor_by_name('import/greedy:0')
beam = g.get_tensor_by_name('import/beam:0')
test_sess = tf.InteractiveSession(graph = g)

In [51]:
from tensorflow.tools.graph_transforms import TransformGraph

In [52]:
transforms = ['add_default_attributes',
             'remove_nodes(op=Identity, op=CheckNumerics, op=Dropout)',
             'fold_constants(ignore_errors=true)',
             'fold_batch_norms',
             'fold_old_batch_norms',
             'quantize_weights(fallback_min=-10, fallback_max=10)',
             'strip_unused_nodes',
             'sort_by_execution_order']

pb = 'transformer-base/frozen_model.pb'
input_graph_def = tf.GraphDef()
with tf.gfile.FastGFile(pb, 'rb') as f:
    input_graph_def.ParseFromString(f.read())

print(pb)

transformed_graph_def = TransformGraph(input_graph_def, 
                                       ['Placeholder'],
                                       ['greedy', 'beam'], transforms)

with tf.gfile.GFile(f'{pb}.quantized', 'wb') as f:
    f.write(transformed_graph_def.SerializeToString())

transformer-base/frozen_model.pb


2022-07-05 23:50:27.644658: I tensorflow/tools/graph_transforms/transform_graph.cc:318] Applying add_default_attributes
2022-07-05 23:50:27.732321: I tensorflow/tools/graph_transforms/transform_graph.cc:318] Applying remove_nodes
2022-07-05 23:50:27.808636: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for greedy
2022-07-05 23:50:27.808688: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for beam
2022-07-05 23:50:27.919622: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for greedy
2022-07-05 23:50:27.919662: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for beam
2022-07-05 23:50:28.003890: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for greedy
2022-07-05 23:50:28.003936: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for beam
2022-07-05 23:50:28.092432: I tensorflow/tools/graph_transforms/remove_nodes.cc:

In [39]:
string = "Beliau yang juga saksi pendakwaan kesembilan berkata, ia bagi mengelak daripada wujud isu digunakan terhadap Najib."
encoded = encoder.encode(string) + [1]
g = test_sess.run([greedy], feed_dict = {x:[encoded]})

In [43]:
encoder.decode(g[0][0])

'He, who is also the ninth prosecution witness, said it was to avoid any issues being used against Najib.<EOS>'

In [44]:
from malaya_boilerplate.huggingface import upload_dict

In [46]:
!cp t2t-noisy-ms-en/base/*.tfevents.* transformer-base
!tar -cvf transformer-base-noisy-ms-en.tar transformer-base

transformer-base/
transformer-base/checkpoint
transformer-base/events.out.tfevents.1656740328.huseincomel-desktop
transformer-base/events.out.tfevents.1656946953.huseincomel-desktop
transformer-base/events.out.tfevents.1656864889.huseincomel-desktop
transformer-base/model.ckpt.index
transformer-base/events.out.tfevents.1656841158.huseincomel-desktop
transformer-base/model.ckpt.data-00000-of-00001
transformer-base/model.ckpt.meta
transformer-base/frozen_model.pb


In [47]:
files_mapping = {'transformer-base-noisy-ms-en.tar': 'transformer-base-noisy-ms-en.tar'}
upload_dict(model = 'pretrained-translation', files_mapping = files_mapping)

In [48]:
files_mapping = {'transformer-base/frozen_model.pb': 'model.pb'}
upload_dict(model = 'translation-ms-en-noisy-base', files_mapping = files_mapping)

In [53]:
files_mapping = {'transformer-base/frozen_model.pb.quantized': 'model.pb'}
upload_dict(model = 'translation-ms-en-noisy-base-quantized', files_mapping = files_mapping)